### 마이그레이션 시 CPU 호환에 중요한 것으로 판단되는 flag들을 인스턴스 별로 정리

In [1]:
# 코드 자동완성 허용
%config Completer.use_jedi = False

# cell 너비 확장
from IPython.display import display, HTML

display(
    HTML(
        data="""
            <style>
                div#notebook-container      { width: 99%}
                div#menubar-container       { width: 99%}
                div#maintoolbar-container   { width: 99%}
            </style>
        """
    )
)

In [2]:
# import library
from IPython.display import display
import pandas as pd
pd.set_option('display.max_columns', None) # Unlimiting Output Length
pd.set_option('display.max_row', None) # Unlimiting Output Length
pd.set_option('display.max_colwidth', None) # Unlimiting Output Column Width Length

import gspread as gs
from gspread_formatting import *

아래 features는 flag가 features name과 일치하지 않음. 주석을 참고.  
ex) XMM feature는 'sse' 로 표기

```c
#define X86_FEATURE_XMM			( 0*32+25) /* "sse" */
#define X86_FEATURE_XMM2		( 0*32+26) /* "sse2" */
#define X86_FEATURE_XMM3		( 4*32+ 0) /* "pni" SSE-3 */
#define X86_FEATURE_XMM4_1		( 4*32+19) /* "sse4_1" SSE-4.1 */
#define X86_FEATURE_XMM4_2		( 4*32+20) /* "sse4_2" SSE-4.2 */
#define X86_FEATURE_MWAIT		( 4*32+ 3) /* "monitor" MONITOR/MWAIT support */
```

In [3]:
# Define core CPU features of CRIU(simplification)
# Exclude features that exist or do not exist in all instances type

# Intel
CPUID_1_ECX = ['pclmulqdq', 'monitor', 'movbe', 'aes', 'xsave', 'avx', 'f16c', 'rdrand']
CPUID_7_0_EBX = ['fsgsbase', 'bmi1', 'hle', 'avx2', 'bmi2', 'erms', 'rtm', 'mpx', 'avx512f', 'avx512dq', 'rdseed', 'adx', 'clflushopt', 'avx512cd', 'sha_ni', 'avx512bw', 'avx512vl']
CPUID_7_0_ECX = ['avx512vbmi', 'avx512_vbmi2', 'gfni', 'vaes', 'vpclmulqdq', 'avx512_vnni', 'avx512_bitalg', 'tme', 'avx512_vpopcntdq', 'rdpid']

# AMD
CPUID_8000_0001_EDX = [ 'mmxext', 'rdtscp']
CPUID_8000_0001_ECX = ['abm', 'sse4a', 'misalignsse', '3dnowprefetch']
CPUID_8000_0008_EBX = ['clzero']

# Extended state features
CPUID_D_1_EAX = ['xsaveopt', 'xsavec', 'xgetbv1']

# All
CPU_FEATURES = CPUID_1_ECX + CPUID_7_0_EBX + CPUID_7_0_ECX + CPUID_8000_0001_EDX + CPUID_8000_0001_ECX + CPUID_8000_0008_EBX + CPUID_D_1_EAX

In [7]:
# csv read & add header

df = pd.read_csv('../lscpu/CPU(s) visualization.csv', usecols=['CloudProvider', 'Architecture', 'InstanceType', 'Model name', 'Flags'], index_col='InstanceType')
df = df.reset_index()
df = df.loc[df['Architecture'] == 'x86_64']

df.drop('Architecture', axis=1, inplace=True)
df = df[['InstanceType', 'CloudProvider', 'Model name', 'Flags']]

df = pd.concat([df, pd.DataFrame(columns=CPU_FEATURES)], axis=1)

df.head(1)

,InstanceType,CloudProvider,Model name,Flags,pclmulqdq,monitor,movbe,aes,xsave,avx,f16c,rdrand,fsgsbase,bmi1,hle,avx2,bmi2,erms,rtm,mpx,avx512f,avx512dq,rdseed,adx,clflushopt,avx512cd,sha_ni,avx512bw,avx512vl,avx512vbmi,avx512_vbmi2,gfni,vaes,vpclmulqdq,avx512_vnni,avx512_bitalg,tme,avx512_vpopcntdq,rdpid,mmxext,rdtscp,abm,sse4a,misalignsse,3dnowprefetch,clzero,xsaveopt,xsavec,xgetbv1
0,t2d-standard-2,GCP,AMD EPYC 7B13,fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ht syscall nx mmxext fxsr_opt pdpe1gb rdtscp lm constant_tsc rep_good nopl nonstop_tsc cpuid extd_apicid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm cmp_legacy cr8_legacy abm sse4a misalignsse 3dnowprefetch osvw topoext invpcid_single ssbd ibrs ibpb stibp vmmcall fsgsbase tsc_adjust bmi1 avx2 smep bmi2 erms invpcid rdseed adx smap clflushopt clwb sha_ni xsaveopt xsavec xgetbv1 clzero xsaveerptr arat npt nrip_save umip rdpid,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
# 지원하지 않는 인스턴스 제거

unsupported = ['m2.xlarge', 'm2.2xlarge', 'm2.4xlarge', 'm1.large', 'm1.xlarge', 'c3.large', 'r3.large', 'm3.large', 'r3.xlarge', 'c3.xlarge', 'm3.xlarge', 'r3.2xlarge', 'm3.2xlarge', 'c3.2xlarge', 'r3.4xlarge', 'c3.4xlarge', 'r3.8xlarge', 'c3.8xlarge']

for instance in unsupported:
    df = df.drop(df[df['InstanceType'] == instance].index)

In [14]:
# Extract each flag

for i in range(len(df)):
    columnIndex = df.columns.get_loc('Flags')
    flags = df.iloc[i, columnIndex]
    flagList = flags.split(' ')

    for j in range(len(CPU_FEATURES)):
        isExist = CPU_FEATURES[j] in flagList
        columnIndex += 1
        if(isExist):
            df.iat[i, columnIndex] = 1
        else:
            df.iat[i, columnIndex] = 0

df.reset_index(drop=True, inplace=True)

In [15]:
# "Flags" column position change to last column

tempColumn = df.pop('Flags')
df.insert(len(df.columns), 'Flags', tempColumn)

df.head(1)

,InstanceType,CloudProvider,Model name,pclmulqdq,monitor,movbe,aes,xsave,avx,f16c,rdrand,fsgsbase,bmi1,hle,avx2,bmi2,erms,rtm,mpx,avx512f,avx512dq,rdseed,adx,clflushopt,avx512cd,sha_ni,avx512bw,avx512vl,avx512vbmi,avx512_vbmi2,gfni,vaes,vpclmulqdq,avx512_vnni,avx512_bitalg,tme,avx512_vpopcntdq,rdpid,mmxext,rdtscp,abm,sse4a,misalignsse,3dnowprefetch,clzero,xsaveopt,xsavec,xgetbv1,Flags
0,t2d-standard-2,GCP,AMD EPYC 7B13,1,0,1,1,1,1,1,1,1,1,0,1,1,1,0,0,0,0,1,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ht syscall nx mmxext fxsr_opt pdpe1gb rdtscp lm constant_tsc rep_good nopl nonstop_tsc cpuid extd_apicid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm cmp_legacy cr8_legacy abm sse4a misalignsse 3dnowprefetch osvw topoext invpcid_single ssbd ibrs ibpb stibp vmmcall fsgsbase tsc_adjust bmi1 avx2 smep bmi2 erms invpcid rdseed adx smap clflushopt clwb sha_ni xsaveopt xsavec xgetbv1 clzero xsaveerptr arat npt nrip_save umip rdpid


In [16]:
# write google spread sheet1(core features)

gc = gs.service_account(filename='../secure-outpost-380004-8d45b1504f3e.json')

sheet = gc.open('CPU Feature Visualization').worksheet('core features')
sheet.clear() # 이전 데이터 삭제
sheet.update([df.columns.values.tolist()] + df.values.tolist())

format_cell = cellFormat(
    verticalAlignment='MIDDLE', 
    wrapStrategy='OVERFLOW_CELL', 
    textFormat=textFormat(fontSize=10)
)

format_cell_range(sheet, '1:500', format_cell)

{'spreadsheetId': '17V0eoAl-4STj_KoQ5SOf2wRohbt6LxovyQ2HDvIxt-c',
 'replies': [{}]}

In [18]:
# Extract features that exist or do not exist on all instances

zero_cols = df.columns[df.eq(0).all(axis=0)].tolist()
print(zero_cols)
print(f"The number of features not exists in all instances : {len(zero_cols)}\n")

one_cols = df.columns[df.eq(1).all(axis=0)].tolist()

print(one_cols)
print(f"The number of features exists in all instances : {len(one_cols)}\n")

# InstanceType, CloudProvider, Model Name, Flags 제외 총 칼럼 개수 = 적어도 1개 이상 사용되는 CPU features 개수
print(f"Number of CPU features used at least one : {len(df.columns) - 4}")

[]
The number of features not exists in all instances : 0

[]
The number of features exists in all instances : 0

Number of CPU features used at least one : 45
